### Evaluating MMIDAS - Mouse smartseq data
This notebook guides you in evaluating MMIDAS in identifying meaningful cell types.

In [1]:
import warnings
import subprocess
import os
from os import path
import glob
import functools
from typing import Mapping, Any, Literal, Iterable, Sequence, Optional

import numpy as np
import torch as th
from torch import nn
from torch import optim
from torch import cuda
from torch.utils.data import DataLoader
from scipy.optimize import linear_sum_assignment
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
from mpl_toolkits.axes_grid1 import make_axes_locatable
from sklearn.metrics.cluster import adjusted_mutual_info_score
from sklearn.preprocessing import normalize
from tqdm import trange, tqdm


def run_cmd(cmd: str) -> str:
    return subprocess.check_output(cmd, shell=True).decode().strip()


def get_parentdirname(d: str) -> str:
    return path.basename(path.dirname(d))


def is_childdir(d: str, c: str) -> bool:
    return c in os.listdir(d)


def fix_dir(d: str) -> None:
    if os.getcwd().endswith(d):
        pass
    elif get_parentdirname(os.getcwd()) == d:
        os.chdir("..")
    elif is_childdir(os.getcwd(), d):
        os.chdir(d)


def dedup(xs: Iterable) -> Iterable:
    acc = []
    seen = set()
    for x in xs:
        if x not in seen:
            acc.append(x)
        seen.add(x)
    return acc


def tree_reload() -> None:
    raise NotImplementedError


def unstable(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        warnings.warn(
            f"{func.__name__}() is unstable", category=FutureWarning, stacklevel=2
        )
        return func(*args, **kwargs)

    return wrapper


DIR = "distributed-vae"  # warning: change this if you change the name of the directory

fix_dir(DIR)

In [4]:
import mmidas
from mmidas.nn_model import mixVAE_model
from mmidas.utils.tree_based_analysis import get_merged_types
from mmidas.cpl_mixvae import cpl_mixVAE, unwrap, asnp
from mmidas.utils.tools import get_paths
from mmidas.utils.dataloader import load_data, get_loaders
from mmidas.utils.cluster_analysis import K_selection
from mmidas.utils.dataloader import load_data
from mmidas.eval_models import summarize_inference

In [5]:
MouseSmartSeq = Literal["mouse_smartseq"]
Mouse10x = Literal["TODO"]
SeattleAlzheimer = Literal["TODO"]

Dataset = MouseSmartSeq | Mouse10x | SeattleAlzheimer

dataset = "mouse_smartseq"
config: Mapping[str, Any] = get_paths("mmidas.toml", dataset)
data: Mapping[str, Any] = load_data(
    config[dataset]["data_path"] / config[dataset]["anndata_file"]
)
dedup(type(v) for v in data.values())

/allen/programs/celltypes/workgroups/mousecelltypes/Hilal/distributed-vae/mmidas.toml
Getting files directories belong to mouse_smartseq...


/home/hilal.mufti/Hilal/distributed-vae/.venv/lib/python3.12/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


data is loaded!
 --------- Data Summary --------- 
# cell types: 115 | # cells: 22365 | # genes:5032


[numpy.matrix, numpy.ndarray, pandas.core.arrays.categorical.Categorical, int]

In [6]:
def get_trained_models(f: str) -> Sequence[str]:
    return glob.glob(f + "/model/cpl_mixVAE_model_before**")


trained_model_folder = config[dataset]["trained_model"]
saving_folder = str(
    config["paths"]["main_dir"] / config[dataset]["saving_path"] / trained_model_folder
)
trained_models = get_trained_models(saving_folder)
selected_model = trained_models[0]
selected_model

'/allen/programs/celltypes/workgroups/mousecelltypes/Hilal/distributed-vae/mmidas-results/K92_S2_AUGTrue_LR0.001_A3_B5000_E100000_Ep0_RUN0/model/cpl_mixVAE_model_before_pruning_A3_2024-09-11-16-59-27.pth'

##### Construct a cpl-mixVAE object and load the trained_model.

In [7]:
# TODO: just parse the file name to get these params

n_categories = 92  # upper bound of number of categories (clusters)
state_dim = 2  # continuous (state) variable dimensionality
n_arm = 3  # number of arms
latent_dim = 10  # latent dimensionality of the model
train_loader, test_loader, all_loader = get_loaders(
    dataset=data["log1p"], batch_size=5000, seed=546
)

# cplMixVAE = cpl_mixVAE(saving_folder=saving_folder, device='cpu')
# cplMixVAE.init_model(n_categories=n_categories,
#                      state_dim=state_dim,
#                      input_dim=data['log1p'].shape[1],
#                      lowD_dim=latent_dim,
#                      n_arm=n_arm)
# cplMixVAE.variational = False

In [8]:
from importlib import reload

reload(mmidas.cpl_mixvae)
reload(mmidas.nn_model)
from mmidas.cpl_mixvae import cpl_mixVAE

trainer = cpl_mixVAE(saving_folder=saving_folder, device="cuda")
trainer.init_model(
    n_categories=n_categories,
    state_dim=state_dim,
    input_dim=data["log1p"].shape[1],
    lowD_dim=latent_dim,
    n_arm=n_arm,
)

using cuda: NVIDIA A100-PCIE-40GB


## Model selection

##### Determining the sub-optimal number of clusters by leveraging consensus values across multiple pruning runs.
If you do not employ the pruning algorithm to determine the number of clusters, you can skip the following step.

In [ ]:
# Loading trained models including before pruning and after pruning
summary = summarize_inference(cplMixVAE, trained_models, train_loader)
summary.keys()

In [ ]:
# Plotting average consensus measure to select the number of clusters according to the minimum consensus measure, here 0.95

summary_file = saving_folder + f"/summary_performance_K_{n_categories}_narm_{n_arm}.p"
with open(summary_file, "rb") as f:
    summary_dict = np.load(f, allow_pickle=True)
f.close()

model_order = K_selection(summary_dict, n_categories, n_arm, thr=0.95)

Assessing the performance of the model for the chosen model order, which denotes the dimensionality of the categorical representation in the mixture VAE model.

In [ ]:
# selected_model = glob.glob(saving_folder + '/model/cpl_mixVAE_model_after_pruning_' + str(n_categories - model_order) + '*')[0]
model_order = n_categories
selected_model = glob.glob(saving_folder + "/model/cpl_mixVAE_model_before_**")[0]
outcome = summarize_inference(cplMixVAE, selected_model, train_loader)

In [25]:
Run = str | int
Config = Mapping[str, Any]
TrainedModelFile = str
Summary = Mapping[Any, Any]
Arm = int
Arms = int

Consensus = float
MI = float

TrainedModel = nn.Module

Bias = th.Tensor
PruningMask = th.Tensor | np.ndarray
PruningIndex = th.Tensor

MixVAE = mixVAE_model


def mk_vae(
    C,
    state_dim,
    input_dim,
    device,
    eps=1e-8,
    fc_dim=100,
    latent_dim=10,
    x_drop=0.5,
    s_drop=0.2,
    lr=0.001,
    lam=1,
    lam_pc=1,
    A=2,
    tau=0.005,
    beta=1.0,
    hard=False,
    variational=True,
    ref_prior=False,
    momentum=0.01,
    mode="MSE",
) -> nn.Module:
    return mixVAE_model(
        input_dim=input_dim,
        fc_dim=fc_dim,
        n_categories=C,
        state_dim=state_dim,
        lowD_dim=latent_dim,
        x_drop=x_drop,
        s_drop=s_drop,
        n_arm=A,
        lam=lam,
        lam_pc=lam_pc,
        tau=tau,
        beta=beta,
        hard=hard,
        variational=variational,
        device=device,
        eps=eps,
        ref_prior=ref_prior,
        momentum=momentum,
        loss_mode=mode,
    ).to(device)


def _mk_vae_cfg(A: int) -> Mapping[str, Any]:
    return {
        "C": 92,
        "state_dim": 2,
        "input_dim": data["log1p"].shape[1],
        "device": "cuda",
        "A": A,
        "latent_dim": 10,
}


def unwrap_literal(x: Dataset) -> str:
    return x.__args__[0]


def mk_run(arms: int, run: int, epochs: int = 100000) -> Run:
    if arms == 5:
        return f"K92_S2_AUGTrue_LR0.001_A{arms}_B5000_E200000_Ep0_RUN{run}"
    else:
        return f"K92_S2_AUGTrue_LR0.001_A{arms}_B5000_E{epochs}_Ep0_RUN{run}"


def mk_config(r: Run, d: Dataset) -> Config:
    config = get_paths("mmidas.toml", unwrap_literal(d))
    config["mouse_smartseq"]["trained_model"] = r
    return config


def get_weights(r: Run, d: Dataset) -> TrainedModelFile:
    c: Config = mk_config(r, d)
    saving_folder = c["paths"]["main_dir"] / c[unwrap_literal(d)]["saving_path"]
    trained_model_folder = c[unwrap_literal(d)]["trained_model"]
    saving_folder = str(saving_folder / trained_model_folder)
    trained_models = glob.glob(saving_folder + "/model/cpl_mixVAE_model_before**")
    assert len(trained_models) == 1
    return trained_models[0]

def view_weights(arms: int, run: int) -> None:
    return th.load(get_weights(mk_run(arms, run), MouseSmartSeq), map_location="cpu")["model_state_dict"]


def load_weights(m: nn.Module, f: str) -> None:
    m.load_state_dict(th.load(f, map_location="cpu")["model_state_dict"])


def load_vae(arms: int, run: int) -> MixVAE:
    r = mk_run(arms, run)
    vae = mk_vae(**_mk_vae_cfg(arms))
    print(get_weights(r, MouseSmartSeq))
    load_weights(vae, get_weights(r, MouseSmartSeq))
    return vae


@unstable
def mk_masks(bias: th.Tensor) -> tuple[np.ndarray, np.ndarray]:
    return np.where(bias.cpu() != 0)[0], np.where(bias.cpu() == 0)[0]


@unstable
def generate(f: nn.Module, dl: DataLoader) -> Mapping[str, Any]:
    A = f.n_arm
    C = f.n_categories
    N = len(dl.dataset)
    D = f.input_dim
    D_low = f.lowD_dim
    S = f.state_dim
    B = unwrap(dl.batch_size)
    dev = f.device

    pruning_mask, inds_prune = mk_masks(f.fcc[0].bias)
    x_recs = np.zeros((A, N, D))
    s_means = np.zeros((A, N, S))
    s_logvars = np.zeros((A, N, S))
    cs = np.zeros((A, N, C))
    c_smps = np.zeros((A, N, C))
    x_lows = np.zeros((A, N, D_low))
    inds_x = np.zeros(N)
    losses = []
    c_dists = []
    c_l2_dists = []
    loss_recs = [[] for _ in range(A)]
    lls = [[] for _ in range(A)]
    preds = np.zeros((A, N))
    f.eval()
    with th.no_grad():
        for i, (x, i_x) in tqdm(enumerate(dl), total=len(dl)):
            x, i_x = x.to(dev), i_x.to(int)
            n_fst, n_lst = i * B, min((i + 1) * B, N)

            xs = [x for _ in range(A)]
            _x_recs, p_xs, r_xs, _x_lows, _cs, _, _c_smps, _s_means, _s_logvars, _ = f(
                xs, temp=1.0, prior_c=0, eval=True, mask=pruning_mask
            )
            _loss, _loss_recs, _, _, _c_dists, _c_l2_dists, _, _, _lls = f.loss(
                _x_recs, p_xs, r_xs, xs, _s_means, _s_logvars, _cs, _c_smps, 0
            )
            losses.append(_loss.item())
            c_dists.append(_c_dists.item())
            c_l2_dists.append(_c_l2_dists.item())
            for a, (loss_rec, ll) in enumerate(zip(_loss_recs, _lls)):
                loss_recs[a].append(loss_rec.item())
                lls[a].append(ll.item())
            outs = map(
                lambda ys: map(asnp, ys),
                zip(_s_means, _s_logvars, _cs, _c_smps, _x_lows, _x_recs),
            )
            for a, (s_mean, s_logvar, c, c_smp, x_low, x_rec) in enumerate(outs):
                s_means[a, n_fst:n_lst, :] = s_mean
                s_logvars[a, n_fst:n_lst, :] = s_logvar
                cs[a, n_fst:n_lst, :] = c
                c_smps[a, n_fst:n_lst, :] = c_smp
                x_lows[a, n_fst:n_lst, :] = x_low
                x_recs[a, n_fst:n_lst, :] = x_rec
                inds_x[n_fst:n_lst] = i_x.cpu().numpy().astype(int)
                preds[a, n_fst:n_lst] = np.argmax(c, axis=-1) + 1
    return {
        "x_recs": x_recs,
        "s_means": s_means,
        "s_logvars": s_logvars,
        "cs": cs,
        "c_smps": c_smps,
        "x_lows": x_lows,
        "inds_x": inds_x,
        "losses": losses,
        "c_dists": np.mean(np.array(c_dists)),
        "c_l2_dists": np.mean(np.array(c_l2_dists)),
        "loss_recs": np.array([np.mean(np.array(loss_recs[a])) for a in range(A)]),
        "lls": np.array([np.mean(np.array(lls[a])) for a in range(A)]),
        "inds_prune": inds_prune,
        "pruning_mask": pruning_mask,
        "preds": preds,
    }

In [42]:
view_weights(5, 2)

/tmp/ipykernel_108774/235498185.py:103: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return th.load(get_weights(mk_run(arms, run), MouseSmartSeq), map_location="cpu")["mode

OrderedDict([('fc1.0.weight',
              tensor([[-8.4274e-04,  4.6375e-03, -4.7201e-03,  ...,  2.0066e-02,
                       -1.0043e-02, -1.6470e-02],
                      [ 5.6705e-01, -1.6044e+00,  1.6572e+00,  ...,  6.5064e-01,
                        6.7636e-01,  1.0418e+00],
                      [-8.1481e-03,  1.0419e-02,  3.0384e-03,  ..., -5.2709e-03,
                       -3.4280e-03, -1.3430e-02],
                      ...,
                      [ 2.0639e-01,  6.6434e-01, -4.7349e-01,  ...,  2.9353e+00,
                        2.1813e+00,  5.1822e-01],
                      [-1.3558e-01,  2.1197e-01,  4.1197e-01,  ...,  3.3792e+00,
                       -2.4702e+00, -6.5671e-01],
                      [-1.4418e-01, -1.1727e+00, -1.9422e-01,  ..., -3.6328e+00,
                       -2.8355e-01, -2.0202e+00]])),
             ('fc1.0.bias',
              tensor([-1.3713e-02,  2.8251e+00, -2.4484e-03, -3.5474e+00, -2.0507e+00,
                      -9.9515e-01, -2.3

In [16]:
@unstable
def evals(f: nn.Module, dl: DataLoader) -> Mapping[str, Any]:
    C = f.n_categories
    outs = generate(f, dl)

    preds = outs["preds"]
    inds_prune = outs["inds_prune"]
    consensus = []
    consensus_min = []
    consensus_mean = []
    pm = []
    for a, pred_a in tqdm(enumerate(preds)):
        for pred_b in preds[a + 1 :]:
            _pm = np.zeros((C, C))  # performance matrix for arm a vs arm b
            for samp_a, samp_b in zip(pred_a, pred_b):
                _pm[samp_a.astype(int) - 1, samp_b.astype(int) - 1] += 1

            smp_cts = []
            for c in range(C):
                smp_cts.append(max(_pm[c].sum(), _pm[:, c].sum()))
            smp_cts = np.array(smp_cts)

            inds_unpruned = np.where(np.isin(range(C), inds_prune) == False)[0]
            _consensus = np.divide(
                _pm, smp_cts, out=np.zeros_like(_pm), where=smp_cts != 0
            )[:, inds_unpruned][inds_unpruned]  # check this

            consensus.append(_consensus)
            consensus_min.append(np.min(np.diag(_consensus)))
            consensus_mean.append(
                1.0 - ((np.abs(preds[0] - preds[1]) > 0.0).sum() / preds.shape[1])
            )
            pm.append(_pm[inds_unpruned][:, inds_unpruned])

    return {
        "c_dists": outs["c_dists"],
        "c_l2_dists": outs["c_l2_dists"],
        "preds": outs["preds"],
        "inds_prune": outs["inds_prune"],
        "s_means": outs["s_means"],
        "s_logvars": outs["s_logvars"],
        "inds_x": outs["inds_x"],
        "cs": outs["cs"],
        "x_recs": outs["x_recs"],
        "x_lows": outs["x_lows"],
        "consensus": consensus,
        "consensus_min": consensus_min,
        "consensus_mean": consensus_mean,
        "pm": pm,
        "inds_unpruned": inds_unpruned,  # TODO
    }


def evals2(fa: nn.Module, fb: nn.Module, dl: DataLoader) -> Mapping[str, Any]:
    C = fa.n_categories
    outs_a = generate(fa, dl)
    outs_b = generate(fb, dl)

    preds_a = outs_a["preds"]
    preds_b = outs_b["preds"]
    inds_prune = outs_a["inds_prune"]
    consensus = []
    consensus_min = []
    consensus_mean = []
    pm = []

    consensus_a = []
    consensus_min_a = []
    consensus_mean_a = []
    pm_a = []
    consensus_b = []
    consensus_min_b = []
    consensus_mean_b = []
    pm_b = []
    for a, pred_a in tqdm(enumerate(preds_a), total=len(preds_a)):
        for pred_b in preds_b:
            _pm = np.zeros((C, C))
            for samp_a, samp_b in zip(pred_a, pred_b):
                _pm[samp_a.astype(int) - 1, samp_b.astype(int) - 1] += 1

            smp_cts = []
            for c in range(C):
                smp_cts.append(max(_pm[c].sum(), _pm[:, c].sum()))
            smp_cts = np.array(smp_cts)

            inds_unpruned = np.where(np.isin(range(C), inds_prune) == False)[0]
            _consensus = np.divide(
                _pm, smp_cts, out=np.zeros_like(_pm), where=smp_cts != 0
            )[:, inds_unpruned][inds_unpruned]

            consensus.append(_consensus)
            consensus_min.append(np.min(np.diag(_consensus)))
            consensus_mean.append(
                1.0 - ((np.abs(preds_a[0] - preds_b[0]) > 0.0).sum() / preds_a.shape[1])
            )
            pm.append(_pm[inds_unpruned][:, inds_unpruned])

        for pred_b in preds_a[a + 1 :]:
            _pm = np.zeros((C, C))
            for samp_a, samp_b in zip(pred_a, pred_b):
                _pm[samp_a.astype(int) - 1, samp_b.astype(int) - 1] += 1

            smp_cts = []
            for c in range(C):
                smp_cts.append(max(_pm[c].sum(), _pm[:, c].sum()))
            smp_cts = np.array(smp_cts)

            inds_unpruned = np.where(np.isin(range(C), inds_prune) == False)[0]
            _consensus = np.divide(
                _pm, smp_cts, out=np.zeros_like(_pm), where=smp_cts != 0
            )[:, inds_unpruned][inds_unpruned]

            consensus_a.append(_consensus)
            consensus_min_a.append(np.min(np.diag(_consensus)))
            consensus_mean_a.append(
                1.0 - ((np.abs(preds_a[0] - preds_a[1]) > 0.0).sum() / preds_a.shape[1])
            )
            pm_a.append(_pm[inds_unpruned][:, inds_unpruned])

    for a, pred_a in tqdm(enumerate(preds_b), total=len(preds_b)):
        for pred_b in preds_b[a + 1 :]:
            _pm = np.zeros((C, C))
            for samp_a, samp_b in zip(pred_a, pred_b):
                _pm[samp_a.astype(int) - 1, samp_b.astype(int) - 1] += 1

            smp_cts = []
            for c in range(C):
                smp_cts.append(max(_pm[c].sum(), _pm[:, c].sum()))
            smp_cts = np.array(smp_cts)

            inds_unpruned = np.where(np.isin(range(C), inds_prune) == False)[0]
            _consensus = np.divide(
                _pm, smp_cts, out=np.zeros_like(_pm), where=smp_cts != 0
            )[:, inds_unpruned][inds_unpruned]

            consensus_b.append(_consensus)
            consensus_min_b.append(np.min(np.diag(_consensus)))
            consensus_mean_b.append(
                1.0 - ((np.abs(preds_b[0] - preds_b[1]) > 0.0).sum() / preds_b.shape[1])
            )
            pm_b.append(_pm[inds_unpruned][:, inds_unpruned])

    return {
        "consensus": consensus,
        "consensus_min": consensus_min,
        "consensus_mean": consensus_mean,
        "pm": pm,
        "consensus_a": consensus_a,
        "consensus_min_a": consensus_min_a,
        "consensus_mean_a": consensus_mean_a,
        "pm_a": pm_a,
        "consensus_b": consensus_b,
        "consensus_min_b": consensus_min_b,
        "consensus_mean_b": consensus_mean_b,
        "pm_b": pm_b,
        "inds_unpruned": inds_unpruned,
        "cs_a": outs_a["cs"],
        "cs_b": outs_b["cs"],
    }


def compare_state_dicts(model1: nn.Module, model2: nn.Module, rtol=1e-5, atol=1e-8):
    state_dict1 = model1.state_dict()
    state_dict2 = model2.state_dict()

    if state_dict1.keys() != state_dict2.keys():
        print("The state dictionaries have different keys.")
        return False

    for key in state_dict1.keys():
        if not th.allclose(state_dict1[key], state_dict2[key], rtol=rtol, atol=atol):
            print(f"Mismatch found in layer: {key}")
            return False

    print("The state dictionaries are identical within the specified tolerance.")
    return True

In [ ]:
def inspect_pth_file(file_path):
    # Load the contents of the .pth file
    data = th.load(file_path)

    print(f"Type of loaded data: {type(data)}")

    if isinstance(data, dict):
        print("\nKeys in the dictionary:")
        for key in data.keys():
            value = data[key]
            print(f"  {key}: {type(value)}")
            if isinstance(value, torch.Tensor):
                print(f"    Shape: {value.shape}, Dtype: {value.dtype}")
    elif isinstance(data, torch.nn.Module):
        print("\nThis is a saved model. State dict keys:")
        for key, value in data.state_dict().items():
            print(f"  {key}: {type(value)}")
            if isinstance(value, torch.Tensor):
                print(f"    Shape: {value.shape}, Dtype: {value.dtype}")
    else:
        print("\nUnexpected data type. Contents:")
        print(data)


In [ ]:
# TODO: why are my weights the same across runs?

A = 2
B = 2
vae_a = load_vae(A, 0)
vae_b = load_vae(B, 1)

# ev = evals(vae_30, all_loader)
ev = evals2(vae_a, vae_b, all_loader)
len(ev["consensus"]), len(ev["consensus_a"]), len(ev["consensus_b"])

In [ ]:
def get_axis(axes: tuple[Arm, Arms, Optional[Run]]) -> str:
    if axes[0] == "ttypes" and axes[1] == "ttypes":
        return "Ttypes classification"
    elif axes[2]:
        return f"Arm {axes[0]} of {axes[1]}-arm MMIDAS, run {axes[2]}"
    else:
        return f"Arm {axes[0]} of {axes[1]}-arm MMIDAS"


def get_axis_save(axes: tuple[Arm, Arms, Optional[Run]]) -> str:
    if axes[0] == "ttypes" and axes[1] == "ttypes":
        return "tt"
    elif axes[2]:
        return f"{axes[0]}{axes[1]}{axes[2]}"
    else:
        return f"{axes[0]}{axes[1]}"


def plot_consensus_circ(
    a_vs_b: np.ndarray,
    axes: tuple[tuple[Arms, Arm, Optional[Run]], tuple[Arms, Arm, Optional[Run]]],
    savedir: Optional[str],
) -> None:
    assert a_vs_b.shape[0] == a_vs_b.shape[1]

    mtx = a_vs_b / np.max(a_vs_b)
    C = mtx.shape[0]  # number of clusters

    fig, axs = plt.subplots(1, 1, figsize=(10, 10))
    for l in trange(C):
        for col in range(C):
            axs.add_patch(
                plt.Circle(np.array([col, l]), radius=mtx[l, col], color="Navy")
            )
    axs.set_xlim([-0.5, C])
    axs.set_ylim([-0.5, C - 0.5])
    axs.invert_yaxis()
    axs.set_yticks([])
    axs.set_xticks([])
    axs.set_yticklabels([])
    axs.set_xticklabels([])
    plt.title("Consensus for |c|= " + str(a_vs_b.shape[0]), fontsize=24)
    axs.set_xlabel(get_axis(axes[0]), fontsize=20)
    axs.set_ylabel(get_axis(axes[1]), fontsize=20)
    fig.tight_layout()
    if savedir:
        plt.savefig(
            savedir
            + f"/consensus_circ_{get_axis_save(axes[0])}_vs_{get_axis_save(axes[1])}_K{C}.png",
            dpi=600,
        )


so_far = 0
so_far_a = 0
so_far_b = 0
for a in range(A):
    for b in range(B):
        plot_consensus_circ(
            ev["pm"][so_far], ((a, A, None), (b, B, None)), f"multiarm-results/{A}{B}"
        )
        so_far += 1

    for b in range(a + 1, A):
        plot_consensus_circ(
            ev["pm_a"][so_far_a],
            ((a, A, None), (b, A, None)),
            f"multiarm-results/{A}{B}",
        )
        so_far_a += 1

for a in range(B):
    for b in range(a + 1, B):
        plot_consensus_circ(
            ev["pm_b"][so_far_b],
            ((a, B, None), (b, B, None)),
            f"multiarm-results/{A}{B}",
        )
        so_far_b += 1

In [ ]:
ConsensusMatrix = np.ndarray


def plot_consensus_heatmap(
    cm: ConsensusMatrix,
    axes: tuple[tuple[Arm, Arms, Optional[Run]], tuple[Arm, Arms, Optional[Run]]],
    savedir: Optional[str],
) -> None:
    plt.figure(figsize=[10, 10])
    ax = plt.gca()
    im = ax.imshow(cm, cmap="rocket", vmin=0, vmax=1)
    plt.xlabel(get_axis(axes[0]), fontsize=30, labelpad=15)
    plt.ylabel(get_axis(axes[1]), fontsize=30, labelpad=15)
    plt.xticks([])
    plt.yticks([])
    plt.title(
        f"Consenus for |c|= {str(cm.shape[0])} (avg={np.mean(np.diag(cm)):.6f})",
        fontsize=32,
        pad=10,
    )
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.3)
    cbar = plt.colorbar(im, cax=cax)
    for t in cbar.ax.get_yticklabels():
        t.set_fontsize(20)

    plt.tight_layout()
    if savedir:
        plt.savefig(
            savedir
            + f"/consensus_heat_{get_axis_save(axes[0])}_vs_{get_axis_save(axes[1])}_K{cm.shape[0]}.png",
            dpi=600,
        )
    print(f"avg consensus on test cells: {np.mean(np.diag(cm))}")


so_far = 0
so_far_a = 0
so_far_b = 0
for a in range(A):
    for b in range(B):
        plot_consensus_heatmap(
            ev["consensus"][so_far],
            ((a, A, None), (b, B, None)),
            f"multiarm-results/{A}{B}",
        )
        so_far += 1

    for b in range(a + 1, A):
        plot_consensus_heatmap(
            ev["consensus_a"][so_far_a],
            ((a, A, None), (b, A, None)),
            f"multiarm-results/{A}{B}",
        )
        so_far_a += 1

for a in range(B):
    for b in range(a + 1, B):
        plot_consensus_heatmap(
            ev["consensus_b"][so_far_b],
            ((a, B, None), (b, B, None)),
            f"multiarm-results/{A}{B}",
        )
        so_far_b += 1

# maybe check images the model got wrong to see what features got wrong
# how much noise to give?

#### Evaluating the model results against a taxonomy
It necessitate access to a taxonomy for the dataset. Some of the subsequent steps depend on this taxonomy.

In [ ]:
# Loading the taxonomy tree and ordering the clusters according to the tree
htree_file = config[dataset]["data_path"] / config[dataset]["htree_file"]
data["cluster"] = np.array([c.strip() for c in data["cluster"]])
cats_ttypes = np.unique(data["cluster"])
C_ttypes = len(cats_ttypes)
merged_cells_labels, treeobj, _ = get_merged_types(
    htree_file=htree_file, cells_labels=data["cluster"], num_classes=1
)

cluster_ids = []
for i, s in enumerate(treeobj.child):
    s = s.strip()
    if (cats_ttypes == s).any():
        cluster_id = list(cats_ttypes).index(s)
        cluster_ids.append(cluster_id)

cats_ttypes_sorted = cats_ttypes[cluster_ids]

inds_ttypes_sorted = []
for s in cats_ttypes_sorted:
    inds_ttypes_sorted.append(list(cats_ttypes).index(s))
inds_ttypes_sorted = np.array(inds_ttypes_sorted)

# Plotting the taxonomy tree
treeobj.plot(
    figsize=[30, 10],
    txtleafonly=True,
    skeletononly=False,
    fontsize=18,
    skeletoncol="gray",
)

In [13]:
# Assign colors to each cell type according to the taxonomy
color_ord_T = treeobj.col[treeobj.isleaf]
color_T = treeobj.col[treeobj.isleaf]
data["cluster_color"] = np.array([color_ord_T[0]] * len(data["cluster"]))
for i_type, ttype in enumerate(cats_ttypes):
    idx1 = np.where(data["cluster"] == ttype)[0]
    idx2 = np.where(cats_ttypes_sorted == ttype)[0][0]
    data["cluster_color"][idx1] = color_ord_T[idx2]
    color_T[i_type] = color_ord_T[idx2]

In [ ]:
outcome = summarize_inference(cplMixVAE, selected_model, all_loader)
category_vs_class = np.zeros((n_arm, data["n_type"], n_categories))

for a in range(A):
    label_predict = []
    for d in range(len(data["cluster_id"])):
        z_cat = np.squeeze(outcome["c_prob"][a][d, :])
        category_vs_class[a, int(data["cluster_id"][d] - 1), np.argmax(z_cat)] += 1
cT_vs_cT = category_vs_class[:, :, outcome["nprune_indx"]]

#### t-types vs. MMIDAS categories

In [ ]:
PerformanceMatrix = np.ndarray
ConsensusMatrix = np.ndarray


def rev(x: np.ndarray) -> np.ndarray:
    return x[::-1]


@unstable
def compute_consensus(pm: np.ndarray, target_dim: int) -> ConsensusMatrix:
    C = pm.shape[-1]

    cat_inds = linear_sum_assignment(-pm)[1]
    sort_idx = np.vstack(
        [rev(np.argsort(pm[c_t])) for c_t in range(target_dim)]
    ).astype(int)
    max_c_ind = -1 * np.ones(C)
    if C > target_dim:
        for c in filter(lambda c: c not in sort_idx[:, 0], range(C)):
            col = np.argmax(pm[:, c].astype(int))
            find_indx = np.where(sort_idx[col, 0] == sort_idx[:, 0])[0][0]
            max_c_ind[find_indx + 1] = c
        max_c_ind[max_c_ind == -1] = sort_idx[:, 0]
        cat_inds = max_c_ind.astype(int)
    return pm[:, cat_inds], cat_inds


def normalize_consensus(pm: np.ndarray) -> ConsensusMatrix:
    return pm / np.max(pm) / 2


@unstable
def plot_consensus(
    cm: np.ndarray,
    xticks,
    yticks,
    colors,
    axes: tuple[tuple[Arm, Arms, Optional[Run]], tuple[Arm, Arms, Optional[Run]]],
    savedir: Optional[str],
) -> None:
    C_targets = cm.shape[0]
    C = cm.shape[1]

    fig, axs = plt.subplots(1, 1, figsize=(12, 15), dpi=100)
    for c_t in trange(C_targets):
        for c in range(C):
            axs.add_patch(
                plt.Circle(np.array([c, c_t]), radius=(cm[c_t, c]), color=colors[c_t])
            )
    axs.set_xlim([-0.1, C + 0.1])
    axs.set_ylim([-0.1, C_targets + 0.1])
    axs.invert_yaxis()
    axs.set_xlabel(f"Categories for {get_axis(axes[0])}", fontsize=20)
    axs.set_xticks(np.arange(C))
    axs.set_xticklabels(xticks, fontsize=6, rotation=90)
    axs.set_yticks(np.arange(C_targets))
    axs.set_yticklabels(yticks, fontsize=8)
    fig.tight_layout()
    if savedir:
        plt.savefig(
            savedir
            + f"/Taxonomy_{get_axis_save(axes[0])}_vs_{get_axis_save(axes[1])}.png",
            dpi=300,
        )
    plt.show()


for a in range(A):
    pm, cat_inds = compute_consensus(
        cT_vs_cT[a, inds_ttypes_sorted, :], len(cats_ttypes_sorted)
    )
    plot_consensus(
        normalize_consensus(pm),
        _ev["inds_unpruned"][cat_inds],
        cats_ttypes_sorted,
        color_ord_T,
        ((a, A, None), ("ttypes", "ttypes", None)),
        saving_folder,
    )

#### Mutual Information between t-types and MMIDAS categories

In [ ]:
def one_hot(probs: np.ndarray) -> np.ndarray:
    return np.eye(probs.shape[-1])[np.argmax(probs, axis=-1)]


def avg_mi(xs: np.ndarray) -> float:
    return np.mean(np.max(xs, axis=-1)).item()


@unstable
def compute_mi(probs: np.ndarray, targets: np.ndarray) -> MI:
    C_pred = probs.shape[-1]
    C_target = targets.shape[-1]

    preds = np.argmax(probs, axis=-1)
    mi = np.zeros((C_target, C_pred))
    for c_p in trange(C_pred):
        for c_t in range(C_target):
            mi[c_t, c_p] = adjusted_mutual_info_score(
                targets[:, c_t], (preds == c_p).astype(int)
            )
    return mi


def normalize_mi(mi: np.ndarray) -> np.ndarray:
    return normalize(mi, axis=1, norm="l1")


def plot_mi(
    mi: np.ndarray,
    axes: tuple[tuple[Arm, Arms], tuple[Arm, Arms]],
    xticks,
    yticks,
    savedir: Optional[str],
) -> None:
    _, col_inds = linear_sum_assignment(-mi)

    fig, ax = plt.subplots(1, 1, figsize=(10, 10), dpi=100)
    plt.title(
        f"Mutual information for |c|= {mi.shape[1]} (avg={avg_mi(mi):.6f})", fontsize=20
    )
    sns.set_theme(font_scale=1.0)
    if xticks and yticks:
        sns.heatmap(
            mi[:, col_inds],
            xticklabels=xticks[col_inds],
            yticklabels=yticks,
            vmin=0,
            vmax=1,
            ax=ax,
            cbar_kws={"shrink": 1},
        )  # pay attention to xticklabels
    else:
        sns.heatmap(mi[:, col_inds], vmin=0, vmax=1, ax=ax, cbar_kws={"shrink": 1})
    ax.set_xlabel(get_axis(axes[0]), fontsize=20)
    ax.set_xticks([])
    ax.set_xticklabels(ax.get_xticklabels(), fontsize=8)
    ax.set_ylabel(get_axis(axes[1]), fontsize=20)
    ax.set_yticks(np.arange(len(yticks)))
    ax.set_yticklabels(yticks, fontsize=8)
    fig.tight_layout()
    if savedir:
        plt.savefig(
            savedir
            + f"/mutinfo_{get_axis_save(axes[0])}_vs_{get_axis_save(axes[1])}_K{mi.shape[1]}.png",
            dpi=300,
        )  # TODO
    plt.show()

def plot_mi_scatter():
    raise NotImplementedError


so_far = 0
so_far_a = 0
so_far_b = 0
for a in range(A):
    for b in range(B):
        mi = compute_mi(ev["cs_a"][a], one_hot(ev["cs_b"][b]))
        plot_mi(
            normalize_mi(mi),
            ((a, A, None), (b, B, None)),
            [],
            [],
            f"multiarm-results/{A}{B}",
        )
        so_far += 1

    for b in range(a + 1, A):
        mi = compute_mi(ev["cs_a"][a], one_hot(ev["cs_a"][b]))
        plot_mi(
            normalize_mi(mi),
            ((a, A, None), (b, A, None)),
            [],
            [],
            f"multiarm-results/{A}{B}",
        )
        so_far_a += 1

for a in range(B):
    for b in range(a + 1, B):
        mi = compute_mi(ev["cs_b"][a], one_hot(ev["cs_b"][b]))
        plot_mi(
            normalize_mi(mi),
            ((a, B, None), (b, B, None)),
            [],
            [],
            f"multiarm-results/{A}{B}",
        )
        so_far_b += 1

In [48]:
def plot_state():
    raise NotImplementedError
    _, data_index = all_loader.dataset.tensors
    data_index = data_index.numpy().astype(int)

    for arm in range(n_arm):
        print(arm)
        plt.close("all")
        fig = plt.figure(figsize=(5, 5))
        m_size = 3
        alp = 0.5
        fontsize = 18

        sns.set_theme()
        sns.set(rc={"axes.facecolor": "whitesmoke"})

        if state_dim == 1:
            axs = fig.add_subplot(1, 1, 1)
            axs.hist(
                outcome["state_mu"][arm][:, 0],
                color=data["cluster_color"][data_index],
                s=m_size,
                alpha=alp,
            )
            axs.set_xlabel(f"T_s_{0}")
        elif state_dim == 2:
            axs = fig.add_subplot(1, 1, 1)
            axs.scatter(
                outcome["state_mu"][arm][:, 0],
                outcome["state_mu"][arm][:, 1],
                color=data["cluster_color"][data_index],
                s=m_size,
                alpha=alp,
            )
            axs.set_xlabel(r"$s_{T_1}$", fontsize=fontsize)
            axs.set_ylabel(r"$s_{T_2}$", fontsize=fontsize)
            axs.xaxis.set_tick_params(labelsize=10)
            axs.yaxis.set_tick_params(labelsize=10)
        elif state_dim == 3:
            axs = fig.add_subplot(1, 1, 1, projection="3d")
            axs.scatter(
                outcome["state_mu"][arm][:, 0],
                outcome["state_mu"][arm][:, 1],
                outcome["state_mu"][arm][:, 2],
                color=data["cluster_color"][data_index],
                s=m_size,
                alpha=alp,
            )
            axs.set_xlabel(f"T_s_{0}")
            axs.set_ylabel(f"T_s_{1}")
            axs.set_zlabel(f"T_s_{2}")

        axs.set_ylim([-2.5, 3])
        axs.set_xlim([-2.5, 2.5])
        axs.set_title(f"Continuous Representation (arm {arm+1})", fontsize=18, pad=15)
        axs.grid(False)
        fig.tight_layout()
        plt.savefig(saving_folder + f"/state_mu_K_{model_order}_arm_{arm}.png", dpi=600)
        plt.show()